<img title="GitHub Octocat" src='./img/Octocat.jpg' style='height: 60px; padding-right: 15px' alt="Octocat" align="left"> This notebook is part of a GitHub repository: https://github.com/pessini/moby-bikes 
<br>MIT Licensed
<br>Author: Leandro Pessini

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import sys
import os
import joblib

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models & Evaluation
from sklearn.model_selection import KFold

# statsmodel
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import statsmodels.stats as stats

# Boost models
import xgboost as xgb
from sklearn import metrics

# Hyperparameter optimization
from bayes_opt import BayesianOptimization

# Custom objects
sys.path.insert(0, os.path.abspath('../src/'))
import experiment_tracker as et

import time
import warnings
warnings.simplefilter('ignore', FutureWarning)
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [2]:
# ideas_df = pd.read_excel('../documentation/experiment_tracker.xlsx', sheet_name='Ideas')
# experiments_df = pd.read_excel('../documentation/experiment_tracker.xlsx', sheet_name='Experiments')

In [3]:
# creates a new object to keep track of the experiments
experiment_tracker = et.ExperimentTracker()

In [4]:
df_train = pd.read_csv('../data/processed/df_train.csv')
df_test = pd.read_csv('../data/processed/df_test.csv')

In [5]:
df = df_train.copy()
X = df.drop(['count'], axis=1)
y = df.pop('count')
all_columns = list(X.columns)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X.shape

(8760, 22)

In [6]:
test_df = df_test.copy()
X_test = test_df.drop(['count'], axis=1)
y_test = test_df.pop('count')
X_test.shape

(1464, 22)

In [7]:
def get_metrics_to_Experiment(dict_scores = None) -> list:
    if dict_scores is None:
        dict_scores = {}
    rsme = et.Score('RSME', '{:.4f}'.format(dict_scores['train_rsme']), '{:.4f}'.format(dict_scores['val_rsme']))
    mae = et.Score('MAE', '{:.4f}'.format(dict_scores['train_mae']), '{:.4f}'.format(dict_scores['val_mae']))
    return [rsme, mae]

In [8]:
import category_encoders as ce

def preprocessor(predictors: list) -> ColumnTransformer:
    # Setting remainder='passthrough' will mean that all columns not specified in the list of “transformers” 
    #   will be passed through without transformation, instead of being dropped

    ##################### Categorical variables #####################
    all_cat_vars = ['timesofday','dayofweek','holiday','peak','hour','working_day','season','month']
    cat_vars = [categorical_var for categorical_var in all_cat_vars if categorical_var in predictors]

    # categorical variables
    cat_pipe = Pipeline([
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])

    cat_encoder = 'cat', cat_pipe, cat_vars

    ##################### Numerical variables #####################
    all_num_vars = ['rain', 'temp', 'rhum','wdsp','temp_r']
    num_vars = [numerical_var for numerical_var in all_num_vars if numerical_var in predictors]

    num_pipe = Pipeline([
        ('scaler', StandardScaler())
        # ('scaler', MinMaxScaler())
    ])

    num_enconder =  'num', num_pipe, num_vars

    ##################### Ordinal variables #####################
    all_ord_vars = ['wind_speed_group','rainfall_intensity']
    ord_vars = [ordinal_var for ordinal_var in all_ord_vars if ordinal_var in predictors]

    ordinal_cols_mapping = []
    if 'wind_speed_group' in predictors:
        ordinal_cols_mapping.append(
            {"col":"wind_speed_group",    
            "mapping": {
                'Calm / Light Breeze': 0, 
                'Breeze': 1, 
                'Moderate Breeze': 2, 
                'Strong Breeze / Near Gale': 3, 
                'Gale / Storm': 4
            }}
        )

    if 'rainfall_intensity' in predictors:
        ordinal_cols_mapping.append(
            {"col":"rainfall_intensity",    
            "mapping": {
                'no rain': 0, 
                'drizzle': 1, 
                'light rain': 2, 
                'moderate rain': 3, 
                'heavy rain': 4
            }}
        )

    # ordinal variables
    ord_pipe = Pipeline([
        ('ordinal', ce.OrdinalEncoder(mapping=ordinal_cols_mapping))
    ])

    ord_enconder =  'ordinal', ord_pipe, ord_vars
    
    #################################################################################
    
    orig_vars = [var for var in predictors if var not in cat_vars and var not in num_vars and var not in ord_vars]
    orig_enconder = 'pass_vars', 'passthrough', orig_vars
     # ['temp_bin','rhum_bin']
    # ord_pipe = 'passthrough'

    transformers_list = []
    transformers_list.append(cat_encoder) if cat_vars else None
    transformers_list.append(ord_enconder) if ord_vars else None
    transformers_list.append(num_enconder) if num_vars else None
    # transformers_list.append(orig_enconder) if orig_vars else None
    
    return ColumnTransformer(transformers=transformers_list, 
                             remainder='drop')

In [9]:
def summarize_dict(dictionary, function):
    return {k: function(v) for k,v in dictionary.items()}

In [10]:
def kfold_score(params, predictors, X=X, y=y, n_folds=5, verbose=50, early_stopping_rounds=10):
    
    pipe_xgboost = Pipeline([
        ('preprocessor', preprocessor(predictors)),
        ('model', xgb.XGBRegressor(**params))
    ])

    X = X[[c for c in X.columns if c in predictors]]
    cv = KFold(n_splits=n_folds, shuffle=True, random_state=2022)
    scores = {"train_rsme":[],"val_rsme":[],"train_mae":[],"val_mae":[]}

    for n_fold, (train_index, test_index) in enumerate(cv.split(X, y)):
        print('#'*40, f'Fold {n_fold+1} out of {cv.n_splits}', '#'*40)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Xy = xgb.DMatrix(X_train, y_train, enable_categorical=True)
        X_test_transformed = pipe_xgboost['preprocessor'].fit_transform(X_test)
        pipe_xgboost.fit(X_train, y_train,
                         model__eval_set=[(X_test_transformed, y_test)], 
                         model__early_stopping_rounds=early_stopping_rounds,
                         model__verbose=verbose)
        # pipe_xgboost.fit(X_train, y_train)
        # print(pipe_xgboost['model'].evals_result())

        # Predict on training and validation set
        y_pred_train = pipe_xgboost.predict(X_train)
        y_pred_val = pipe_xgboost.predict(X_test)

        # Calculate the RSME and MAE
        # If squared = True returns MSE value, if False returns RMSE value.
        scores['train_rsme'].append(metrics.mean_squared_error(y_train, y_pred_train, squared=False))
        scores['val_rsme'].append(metrics.mean_squared_error(y_test, y_pred_val, squared=False))
        scores['train_mae'].append(metrics.mean_absolute_error(y_train, y_pred_train))
        scores['val_mae'].append(metrics.mean_absolute_error(y_test, y_pred_val))

        print(f"Fold {n_fold+1} - best iteration: {pipe_xgboost['model'].get_booster().best_iteration}\n")

    return summarize_dict(scores, np.mean)

In [11]:
#predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

## Hyperparameters tuning

- eta [default=0.3]
Analogous to learning rate in GBM
Makes the model more robust by shrinking the weights on each step
Typical final values to be used: 0.01-0.2
- min_child_weight [default=1]
Defines the minimum sum of weights of all observations required in a child.
This is similar to min_child_leaf in GBM but not exactly. This refers to min “sum of weights” of observations while GBM has min “number of observations”.
Used to control over-fitting. Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree.
Too high values can lead to under-fitting hence, it should be tuned using CV.
- max_depth [default=6]
The maximum depth of a tree, same as GBM.
Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.
Should be tuned using CV.
Typical values: 3-10
- max_leaf_nodes
The maximum number of terminal nodes or leaves in a tree.
Can be defined in place of max_depth. Since binary trees are created, a depth of ‘n’ would produce a maximum of 2^n leaves.
If this is defined, GBM will ignore max_depth.
- gamma [default=0]
A node is split only when the resulting split gives a positive reduction in the loss function. Gamma specifies the minimum loss reduction required to make a split.
Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
- max_delta_step [default=0]
In maximum delta step we allow each tree’s weight estimation to be. If the value is set to 0, it means there is no constraint. If it is set to a positive value, it can help making the update step more conservative.
Usually this parameter is not needed, but it might help in logistic regression when class is extremely imbalanced.
This is generally not used but you can explore further if you wish.
- subsample [default=1]
Same as the subsample of GBM. Denotes the fraction of observations to be randomly samples for each tree.
Lower values make the algorithm more conservative and prevents overfitting but too small values might lead to under-fitting.
Typical values: 0.5-1
- colsample_bytree [default=1]
Similar to max_features in GBM. Denotes the fraction of columns to be randomly samples for each tree.
Typical values: 0.5-1
- colsample_bylevel [default=1]
Denotes the subsample ratio of columns for each split, in each level.
I don’t use this often because subsample and colsample_bytree will do the job for you. but you can explore further if you feel so.
- lambda [default=1]
L2 regularization term on weights (analogous to Ridge regression)
This used to handle the regularization part of XGBoost. Though many data scientists don’t use it often, it should be explored to reduce overfitting.
- alpha [default=0]
L1 regularization term on weight (analogous to Lasso regression)
Can be used in case of very high dimensionality so that the algorithm runs faster when implemented
- scale_pos_weight [default=1]
A value greater than 0 should be used in case of high class imbalance as it helps in faster convergence.

In [21]:
# Baseline model
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':3,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores = kfold_score(params_xgboost, predictors, n_folds=3)
exp_xgboost = et.Experiment('XGBoost (Baseline)', predictors=predictors, hyperparameters=params_xgboost,
                               score=get_metrics_to_Experiment(dict_scores), notes='Baseline XGBoost')
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 3 ########################################
[0]	validation_0-rmse:4.21537
[50]	validation_0-rmse:2.74378
[60]	validation_0-rmse:2.74738
Fold 1 - best iteration: 50

######################################## Fold 2 out of 3 ########################################
[0]	validation_0-rmse:3.78092
[50]	validation_0-rmse:2.50728
[52]	validation_0-rmse:2.51101
Fold 2 - best iteration: 43

######################################## Fold 3 out of 3 ########################################
[0]	validation_0-rmse:3.96064
[50]	validation_0-rmse:2.65082
[60]	validation_0-rmse:2.65222
Fold 3 - best iteration: 50

--- New Experiment added! ---
ID#: 5415556000 
Algorithm: XGBoost (Baseline) 
Predictors: ['temp', 'rhum', 'dayofweek', 'holiday', 'timesofday', 'wdsp', 'rainfall_intensity', 'peak', 'working_day', 'hour', 'season']
Hyperparameters: {'max_depth': 3, 'seed': 42, 'eval_metric': 'rmse'}
Date: 26/06/2022 11:09:14
Metric: [{ 'metric': RSME, 'trai

In [26]:
predictors = ['temp','rhum','dayofweek', 'holiday','timesofday','wdsp','rainfall_intensity','peak','working_day', 'hour', 'season']

params_xgboost = {'max_depth':9,
                  'eta': 0.01,
                  'n_estimators': 2000,
                  'gamma': 1,
                   'seed': 42,
                   'eval_metric': 'rmse'
                   }

dict_scores = kfold_score(params_xgboost, predictors, n_folds=5, verbose=250, early_stopping_rounds=50)
exp_xgboost = et.Experiment('XGBoost (gamma=0.5)', predictors=predictors, hyperparameters=params_xgboost, 
                            score=get_metrics_to_Experiment(dict_scores), 
                            notes='Changed gamma to 0.5')
experiment_tracker.add_experiment(exp_xgboost)

######################################## Fold 1 out of 5 ########################################
[0]	validation_0-rmse:4.98711
[250]	validation_0-rmse:2.74485
[376]	validation_0-rmse:2.72610
Fold 1 - best iteration: 326

######################################## Fold 2 out of 5 ########################################
[0]	validation_0-rmse:4.92152
[250]	validation_0-rmse:2.70298
[500]	validation_0-rmse:2.66915
[511]	validation_0-rmse:2.66981
Fold 2 - best iteration: 462

######################################## Fold 3 out of 5 ########################################
[0]	validation_0-rmse:4.68695
[250]	validation_0-rmse:2.49219
[387]	validation_0-rmse:2.48008
Fold 3 - best iteration: 338

######################################## Fold 4 out of 5 ########################################
[0]	validation_0-rmse:4.71489
[250]	validation_0-rmse:2.55270
[376]	validation_0-rmse:2.54808
Fold 4 - best iteration: 326

######################################## Fold 5 out of 5 #######################

In [14]:
# experiment_tracker.to_excel('../documentation/experiment_tracker_xgboost.xlsx')
# joblib.dump(pipe_xgboost['model'], '../models/XGBoost.pkl')

In [15]:
%reload_ext watermark
%watermark -a "Leandro Pessini" -n -u -v -iv -w

Author: Leandro Pessini

Last updated: Sat Jun 25 2022

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.3.0

seaborn          : 0.11.1
category_encoders: 2.4.0
statsmodels      : 0.13.2
sklearn          : 1.0.2
numpy            : 1.21.1
pandas           : 1.3.0
sys              : 3.9.6 | packaged by conda-forge | (default, Jul 11 2021, 03:36:15) 
[Clang 11.1.0 ]
joblib           : 1.0.1
xgboost          : 1.4.0
matplotlib       : 3.4.2

Watermark: 2.3.0



> https://pierpaolo28.github.io/blog/blog25/#bayesian-optimization

> https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html

> https://www.kaggle.com/code/neerajmohan/randomforest-model-with-bayesian-optimization/notebook

> https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

<img title="GitHub Mark" src="./img/GitHub-Mark-64px.png" style="height: 32px; padding-right: 15px" alt="GitHub Mark" align="left"> [GitHub repository](https://github.com/pessini/moby-bikes) <br>Author: Leandro Pessini